In [211]:
import pandas as pd
import pyarrow.parquet as pa
import spacy


In [212]:
#imports the load dataset from the Hugging Face DataSet libary
from datasets import load_dataset
#loads the first 10,0000 rows from Parquet files in the variable data
data = load_dataset("parquet", data_files="train-00000-of-00001-090b52ccb189d47a.parquet", split='train[:10000]')
data

Dataset({
    features: ['text'],
    num_rows: 10000
})

In [214]:
import re
#checking if row is title 
def is_title(row):
    return len(row['text'].split()) <= 2
    

In [233]:
def separate_title_and_text(dataset):
    final_rows = []
    current_title = None
    accumulated_text = []

    for row in dataset:
        if is_title(row):  # Identify if the row is a title
            if current_title is not None:
                # Save the previous title and its accumulated text
                final_rows.append({
                    'title': current_title,
                    'text': ' '.join(accumulated_text)
                })
            # Update the current title and reset the accumulated text
            current_title = row['text']
            accumulated_text = []
        else:
            # If it's part of the text, accumulate it
            accumulated_text.append(row['text'])

    # Append the last title and its text if exists
    if current_title is not None:
        final_rows.append({
            'title': current_title,
            'text': ' '.join(accumulated_text)
        })

    return final_rows

wiki_data = separate_title_and_text(data)



In [237]:
# converts lists into Hugging Face Dataset
wiki_data=data.from_list(wiki_data)

In [238]:
import tiktoken
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [239]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [240]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [241]:
# Split text into 400-char chunks with 20-char overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""], 
)